In [2]:
import os
# save using vs code
SCALA_VERSION = '2.12'
SPARK_VERSION = '3.1.3'
# Download Kafka Jar file, this for readStream.format("kafka"), "kafka" is a driver
# kafka driver code is part of Maven Jar file
# https://mvnrepository.com/artifact/org.apache.spark/spark-sql-kafka-0-10_2.12/3.1.3
# pyspark-shell shall download the jar file behind..
os.environ['PYSPARK_SUBMIT_ARGS'] = f'--packages org.apache.spark:spark-sql-kafka-0-10_{SCALA_VERSION}:{SPARK_VERSION} pyspark-shell'

In [3]:
import findspark
findspark.init()

In [4]:
import pyspark


from pyspark.sql import SparkSession
# spark groupBy has default setting for spark.sql.shuffle.partitions as 200
# we set to  4, should NOT be done in production 
spark = SparkSession.builder.master("local[1]")\
                            .config('spark.sql.shuffle.partitions', 4)\
                            .appName("SparkStreamingKafkaCandles").getOrCreate()

22/05/25 01:43:59 WARN Utils: Your hostname, ubuntu-virtual-machine resolves to a loopback address: 127.0.1.1; using 192.168.174.129 instead (on interface ens33)
22/05/25 01:43:59 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/opt/spark-3.1.3-bin-hadoop2.7/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ubuntu/.ivy2/cache
The jars for the packages stored in: /home/ubuntu/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-d6bb808c-a8c3-4d72-8986-67985fb48f5d;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.1.3 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.1.3 in central
	found org.apache.kafka#kafka-clients;2.6.0 in central
	found com.github.luben#zstd-jni;1.4.8-1 in central
	found org.lz4#lz4-java;1.7.1 in central
	found org.xerial.snappy#snappy-java;1.1.8.2 in central
	found org.slf4j#slf4j-api;1.7.30 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.commons#commons-pool2;2.6.2 in central
:: resolution report :: resolve 551ms :: artifacts dl 10ms
	:: modules in use:
	com.github.luben#zstd-jni;1.4.8-1 from central in [default]
	org.apache.commons#commons-pool2;2.6.2 from central

In [5]:
# read from kafka, here spark is consumer for kafka topic called test
# spark streaming works as dataframe/sql
# group.id is consumer group id
# subcribe is kafka topic
# "kafka" driver is not available by default with spark, we need to download it, we did on cell 1

kafkaDf = spark.readStream.format("kafka")\
  .option("kafka.bootstrap.servers", "localhost:9092")\
  .option("subscribe", "stock-ticks")\
  .option("group.id", "stock-ticks-group")\
  .load()

In [6]:
# key and value are binary type, we need to CAST To STRING type
kafkaDf.printSchema()
# timestampType values
# CreateTime:  Timestamp relates to message creation time as set by a Kafka client/producer
# LogAppendTime : Timestamp relates to the time a message was appended to a Kafka log.
# UnknownType

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [7]:
# Kafka value, key are in binary format
# we need to type cast to STRING, 
# we pick only value, timestamp as we need for 10 minute window
ticksDf = kafkaDf.selectExpr("CAST(value AS STRING)", "timestamp")
ticksDf.printSchema() # we get only value as string

root
 |-- value: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)



In [8]:
# echoOnconsole = ticksDf\
#                 .writeStream\
#                 .outputMode("update")\
#                 .format("console")\
#                 .option("truncate", False)\
#                 .start() # start the query. spark will subscribe for data

In [9]:
import pyspark.sql.functions as F
from pyspark.sql.types import StructField, StructType, DoubleType, StringType, LongType, TimestampType

schema = StructType( [
    StructField("symbol", StringType(), True),
    StructField("price", DoubleType(), True),
    StructField("volume", LongType(), True),
    StructField("timestamp", LongType(),  True)
])

In [10]:
# we parse value which is JSON string to kafka struct, 
# this create a new column [replacing existing value which is string] with type
jsonDf = ticksDf.withColumn("value", F.from_json("value", schema))
jsonDf.printSchema()

root
 |-- value: struct (nullable = true)
 |    |-- symbol: string (nullable = true)
 |    |-- price: double (nullable = true)
 |    |-- volume: long (nullable = true)
 |    |-- timestamp: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)



In [11]:
# now extract all the sub fields symbol, price, volume, timestamp into data frame
# exisitng value, timestamp shall be removed. existing timestamp is value produced time, not a stock tick time
# value.timestamp that comes from producer is basically a event time
stockTickDf = jsonDf.select (F.col("value.*")) # extract all sub fields as schema
stockTickDf.printSchema()

root
 |-- symbol: string (nullable = true)
 |-- price: double (nullable = true)
 |-- volume: long (nullable = true)
 |-- timestamp: long (nullable = true)



In [12]:
stockTickDf = stockTickDf.withColumn("traded_value", F.col("price") * F.col("volume"))
stockTickDf.printSchema()

root
 |-- symbol: string (nullable = true)
 |-- price: double (nullable = true)
 |-- volume: long (nullable = true)
 |-- timestamp: long (nullable = true)
 |-- traded_value: double (nullable = true)



In [13]:
# we have data coming in stream, each record has timestamp
# we have milli seconds 1647283268253, the actual seconds Mon Mar 14 2022 14:41:08
# now we have to calculate 1 minute window, sum(volume) up to 1 minute
# after conversions, we take timestamp which is truncatd to minute instead of milli seconds, drop temp columns
stockTickDf = stockTickDf\
                .withColumn("timestampTemp", (F.col("timestamp") / 1000).cast("timestamp"))\
                .withColumn("trade_time", F.date_trunc("minute", F.col("timestampTemp")))\
                .drop("timestamp")\
                .drop("timestampTemp")\
                .withColumnRenamed("trade_time", "timestamp")
               

# echoOnconsole = stockTickDf\
#                 .writeStream\
#                 .outputMode("update")\
#                 .format("console")\
#                 .option("truncate", False)\
#                 .start() # start the query. spark will subscribe for data

In [15]:
# aggregate
# sum (volume), sum(traded_value), candles open, high, low, close [OHLC]
# groupBy by symbol, timestamp

# now add delayed record processing with waterMark
stockTickDf1Min = stockTickDf\
                            .withWatermark("timestamp", "1 seconds")\
                            .groupBy("symbol", F.window("timestamp", "60 seconds"))\
                          .agg( F.sum("volume").alias("volume"),\
                                F.sum("traded_value").alias("traded_value"),\
                                F.max("price").alias("high"),\
                                F.min("price").alias("low"),\
                                F.first("price").alias("first"),\
                                F.last("price").alias("last")\
                              )
stockTickDf1Min.printSchema()
echoOnconsole = stockTickDf1Min\
                .writeStream\
                .outputMode("update")\
                .format("console")\
                .option("truncate", False)\
                .start() # start the query. spark will subscribe for data

# write result to kafka
# we need to publish all the field together as value
# we take all columns, convert to json string
# publish json as value to kafka

stockTickDf1MinKafka = stockTickDf1Min\
                            .selectExpr("to_json(struct(*)) AS value")

stockTickDf1MinKafka\
            .writeStream\
             .format("kafka")\
            .outputMode("update")\
             .option("kafka.bootstrap.servers", "localhost:9092")\
            .option("topic", "candles-1min")\
            .option("checkpointLocation", "file:///tmp/spark3")\
            .start()
            
# use kafka console consumer to subscrbe candles-1min, command is given in 2 nd cell

root
 |-- symbol: string (nullable = true)
 |-- window: struct (nullable = false)
 |    |-- start: timestamp (nullable = true)
 |    |-- end: timestamp (nullable = true)
 |-- volume: long (nullable = true)
 |-- traded_value: double (nullable = true)
 |-- high: double (nullable = true)
 |-- low: double (nullable = true)
 |-- first: double (nullable = true)
 |-- last: double (nullable = true)



22/05/25 02:05:00 WARN StreamingQueryManager: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-cd95347e-4c07-4db2-80b1-a037b3ce3f69. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
22/05/25 02:05:00 WARN StreamingQueryManager: Stopping existing streaming query [id=a8ba6bc4-73eb-4276-97ed-e11d851f45f6, runId=21c507c3-388e-4056-8929-dcbf1ff3c972], as a new run is being started.
22/05/25 02:05:00 WARN Shell: Interrupted while joining on: Thread[Thread-1924,5,main]
java.lang.InterruptedException
	at java.lang.Object.wait(Native Method)
	at java.lang.Thread.join(Thread.java:1252)
	at java.lang.Thread.join(Thread.java:1326)
	at org.apache.hadoop.util.Shell.joinThread(Shell.java:629)
	at org.apache.hadoop.util.Shell.runCommand(Shell.java:580)
	at org.apache.hadoop.util.Shell.run(Shell.java:4

-------------------------------------------
Batch: 25
-------------------------------------------
+------+------------------------------------------+------+------------+----+---+-----+----+
|symbol|window                                    |volume|traded_value|high|low|first|last|
+------+------------------------------------------+------+------------+----+---+-----+----+
|TSLA  |{2022-05-25 02:05:00, 2022-05-25 02:06:00}|40    |200.0       |5.0 |5.0|5.0  |5.0 |
+------+------------------------------------------+------+------------+----+---+-----+----+

-------------------------------------------
Batch: 0
-------------------------------------------
+------+------+------+------------+----+---+-----+----+
|symbol|window|volume|traded_value|high|low|first|last|
+------+------+------+------------+----+---+-----+----+
+------+------+------+------------+----+---+-----+----+

-------------------------------------------
Batch: 26
-------------------------------------------
+------+------+------

-------------------------------------------
Batch: 1
-------------------------------------------
+------+------------------------------------------+------+------------+----+----+-----+----+
|symbol|window                                    |volume|traded_value|high|low |first|last|
+------+------------------------------------------+------+------------+----+----+-----+----+
|RELAXO|{2022-05-25 02:05:00, 2022-05-25 02:06:00}|40    |1040.0      |26.0|26.0|26.0 |26.0|
+------+------------------------------------------+------+------------+----+----+-----+----+

-------------------------------------------
Batch: 27
-------------------------------------------
+------+------------------------------------------+------+------------+----+----+-----+----+
|symbol|window                                    |volume|traded_value|high|low |first|last|
+------+------------------------------------------+------+------------+----+----+-----+----+
|RELAXO|{2022-05-25 02:05:00, 2022-05-25 02:06:00}|40    |10

22/05/25 02:05:12 WARN HDFSBackedStateStoreProvider: The state for version 25 doesn't exist in loadedMaps. Reading snapshot file and delta files if needed...Note that this is normal for the first batch of starting query.
22/05/25 02:05:12 WARN HDFSBackedStateStoreProvider: The state for version 25 doesn't exist in loadedMaps. Reading snapshot file and delta files if needed...Note that this is normal for the first batch of starting query.
22/05/25 02:05:12 WARN HDFSBackedStateStoreProvider: The state for version 25 doesn't exist in loadedMaps. Reading snapshot file and delta files if needed...Note that this is normal for the first batch of starting query.
22/05/25 02:05:12 WARN HDFSBackedStateStoreProvider: The state for version 25 doesn't exist in loadedMaps. Reading snapshot file and delta files if needed...Note that this is normal for the first batch of starting query.


...

-------------------------------------------
Batch: 181
-------------------------------------------
+------+------------------------------------------+------+------------+----+----+-----+----+
|symbol|window                                    |volume|traded_value|high|low |first|last|
+------+------------------------------------------+------+------------+----+----+-----+----+
|MSFT  |{2022-05-25 02:30:00, 2022-05-25 02:31:00}|80    |1090.0      |14.0|13.0|14.0 |13.0|
+------+------------------------------------------+------+------------+----+----+-----+----+

-------------------------------------------
Batch: 206
-------------------------------------------
+------+------------------------------------------+------+------------+----+----+-----+----+
|symbol|window                                    |volume|traded_value|high|low |first|last|
+------+------------------------------------------+------+------------+----+----+-----+----+
|MSFT  |{2022-05-25 02:30:00, 2022-05-25 02:31:00}|80    

-------------------------------------------
Batch: 207
-------------------------------------------
+------+------------------------------------------+------+------------+----+----+-----+----+
|symbol|window                                    |volume|traded_value|high|low |first|last|
+------+------------------------------------------+------+------------+----+----+-----+----+
|TSLA  |{2022-05-25 02:31:00, 2022-05-25 02:32:00}|40    |480.0       |12.0|12.0|12.0 |12.0|
+------+------------------------------------------+------+------------+----+----+-----+----+

-------------------------------------------
Batch: 182
-------------------------------------------
+------+------------------------------------------+------+------------+----+----+-----+----+
|symbol|window                                    |volume|traded_value|high|low |first|last|
+------+------------------------------------------+------+------------+----+----+-----+----+
|TSLA  |{2022-05-25 02:31:00, 2022-05-25 02:32:00}|40    

-------------------------------------------
Batch: 208
-------------------------------------------
+------+------+------+------------+----+---+-----+----+
|symbol|window|volume|traded_value|high|low|first|last|
+------+------+------+------------+----+---+-----+----+
+------+------+------+------------+----+---+-----+----+

-------------------------------------------
Batch: 183
-------------------------------------------
+------+------+------+------------+----+---+-----+----+
|symbol|window|volume|traded_value|high|low|first|last|
+------+------+------+------------+----+---+-----+----+
+------+------+------+------------+----+---+-----+----+



-------------------------------------------
Batch: 209
-------------------------------------------
+------+------------------------------------------+------+------------+----+----+-----+----+
|symbol|window                                    |volume|traded_value|high|low |first|last|
+------+------------------------------------------+------+------------+----+----+-----+----+
|MARUTI|{2022-05-25 02:31:00, 2022-05-25 02:32:00}|40    |1480.0      |37.0|37.0|37.0 |37.0|
+------+------------------------------------------+------+------------+----+----+-----+----+

-------------------------------------------
Batch: 184
-------------------------------------------
+------+------------------------------------------+------+------------+----+----+-----+----+
|symbol|window                                    |volume|traded_value|high|low |first|last|
+------+------------------------------------------+------+------------+----+----+-----+----+
|MARUTI|{2022-05-25 02:31:00, 2022-05-25 02:32:00}|40    

-------------------------------------------
Batch: 210
-------------------------------------------
+------+------------------------------------------+------+------------+----+----+-----+----+
|symbol|window                                    |volume|traded_value|high|low |first|last|
+------+------------------------------------------+------+------------+----+----+-----+----+
|RELAXO|{2022-05-25 02:31:00, 2022-05-25 02:32:00}|30    |480.0       |16.0|16.0|16.0 |16.0|
+------+------------------------------------------+------+------------+----+----+-----+----+

-------------------------------------------
Batch: 185
-------------------------------------------
+------+------------------------------------------+------+------------+----+----+-----+----+
|symbol|window                                    |volume|traded_value|high|low |first|last|
+------+------------------------------------------+------+------------+----+----+-----+----+
|RELAXO|{2022-05-25 02:31:00, 2022-05-25 02:32:00}|30    